# Figuring out tile-priority option

In [ ]:
%matplotlib inline
from matplotlib import pyplot
import geopandas
import pandas

from weave_units import WeaveUnit
from tile_map import Tiling

In [ ]:
ak = geopandas.read_file("../data/imd-auckland-2018.gpkg")

In [ ]:
w1 = WeaveUnit(weave_type = "twill", spacing = 100,  aspect = 0.75,
                    margin = .03, strands = "abc|defg", crs = 2193)

fig = pyplot.figure(figsize = (8, 3))
ax = fig.add_subplot(111)
w1.elements.plot(ax = ax, column = "element_id", edgecolor = "k", linewidth = 0)
w1.tile.plot(ax = ax, facecolor = "#ffffff00", 
             edgecolor = "r", linestyle = "dashed", linewidth = 2)

In [ ]:
weave1 = Tiling(w1, ak, id_var = "DZ2018")
rotated_tiles = weave1.rotated(0)

In [ ]:
# clip to the region extent - they will still be one multipolygon per element_id
tiles = rotated_tiles.clip(ak)
# explode to individual elements (this might be how they should be stored!)
tiles = tiles.explode(ignore_index = True)
# add an ID per element
tiles["tileUID"] = list(range(tiles.shape[0]))
# overlay the region data
tiles = tiles.overlay(ak, keep_geom_type = False)
# add an area attribute
tiles["area"] = tiles.geometry.area

In [ ]:
lookup = tiles.iloc[tiles.groupby("tileUID")["area"].agg(
    pandas.Series.idxmax)][["tileUID", "DZ2018"]]

In [ ]:
tiles = tiles.drop(["DZ2018"], axis = 1)
tiles = tiles.merge(lookup, on = "tileUID")
tiles["diss_var"] = tiles.element_id + tiles.DZ2018.astype(str)
tiles = tiles.dissolve(
    by = "diss_var", as_index = False).drop(["diss_var"], axis = 1)

In [ ]:
fig = pyplot.figure(figsize = (24, 15))
ax = fig.add_subplot(111)

strands = set(tiles.element_id)
cols = [x for x in tiles.columns if "Rank" in x]
cmaps = ("OrRd", "BuPu", "Greys", "BuGn", "YlOrRd", "YlGnBu", "RdPu")
cmaps = ("BrBG", "RdGy", "RdYlBu", "Spectral",
         "PiYG", "PRGn", "PuOr", "viridis")

for strand, col, cmap in zip(strands, cols, cmaps):
    tiles[tiles.element_id == strand].plot(ax = ax, column = col, cmap = cmap, k = 100, scheme = "equal_interval", linewidth = 0)

ak.plot(ax = ax, facecolor = "#00000000", edgecolor = "k")